In [12]:
from reportlab.lib.styles import ParagraphStyle, getSampleStyleSheet
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image
from reportlab.lib.utils import ImageReader
import os
import mne
import json

from utils import *
import matplotlib
from et_qc import *
from eda_qc import *
from lsl_problem import *

In [13]:
subject = 'P5182010'
xdf_filename = f'/Users/camilla.strauss/Desktop/CUNY_Data/Data/sub-{subject}/sub-{subject}_ses-S001_task-CUNY_run-001_mobi.xdf'
video_filename = f'/Users/camilla.strauss/Desktop/CUNY_Data/Data/sub-{subject}/sub-{subject}_task-CUNY_run-001_video.avi'

In [ ]:
whole_mic_df = import_mic_data(xdf_filename)

In [17]:
stim_df = import_stim_data(xdf_filename)

In [15]:
whole_mic_df = mic_df

In [18]:
mic_df = get_event_data(event = 'RestingState', df = whole_mic_df, stim_df = stim_df)

In [19]:
mic_df

,int_array,bytestring,duration,lsl_time_stamp,time


In [20]:
stim_df

,trigger,event,lsl_time_stamp,time
0,200,Onset_Experiment,652877.853621,0.000000
1,1729108775,psychopy_time_stamp,652877.853632,0.000011
2,10,Onset_RestingState,652903.462754,25.609133
3,1729108800,psychopy_time_stamp,652903.462766,25.609144
4,11,Offset_RestingState,653203.463529,325.609908
...,...,...,...,...
256,1729111346,psychopy_time_stamp,655451.996951,2574.143330
257,201,Offset_Experiment,655456.997079,2579.143458
258,1729111351,psychopy_time_stamp,655456.997089,2579.143468
259,4,Bx_input,655456.997128,2579.143507


In [21]:
whole_mic_df

,int_array,bytestring,duration,lsl_time_stamp,time
0,32,b' \x00\x00\x00\x00\x00\x00\x00',0.000000e+00,653249.825089,0.000000
1,37,b'%\x00\x00\x00\x00\x00\x00\x00',5.142097e-10,653249.825112,0.000023
2,37,b'%\x00\x00\x00\x00\x00\x00\x00',1.028419e-09,653249.825135,0.000045
3,43,b'+\x00\x00\x00\x00\x00\x00\x00',1.542629e-09,653249.825158,0.000068
4,39,"b""'\x00\x00\x00\x00\x00\x00\x00""",2.056839e-09,653249.825180,0.000091
...,...,...,...,...,...
100124636,11,b'\x0b\x00\x00\x00\x00\x00\x00\x00',5.148506e-02,655520.270591,2270.445501
100124637,11,b'\x0b\x00\x00\x00\x00\x00\x00\x00',5.148506e-02,655520.270613,2270.445524
100124638,12,b'\x0c\x00\x00\x00\x00\x00\x00\x00',5.148506e-02,655520.270636,2270.445546
100124639,7,b'\x07\x00\x00\x00\x00\x00\x00\x00',5.148506e-02,655520.270659,2270.445569


In [22]:
vars = {}

In [24]:
vars['sampling_rate'], vars['lsl_wav_duration_diff'], vars['num_NaN'], vars['percent_NaN'], vars['quan25'], vars['quan75'], vars['std'], vars['min'], vars['max'] = [0,0,0,0,0,0,0,0,0]


In [28]:
vars['quan75']

np.float64(0.0)

In [27]:
vars['sampling_rate'], vars['lsl_wav_duration_diff'], vars['num_NaN'], vars['percent_NaN'], vars['quan25'], vars['quan75'], vars['std'], vars['min'], vars['max'] = np.zeros(9)
